# Part 4: Disaster Strikes

In [6]:
#q8
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 25821052928 (24.05 GB)
Present Capacity: 12024475943 (11.20 GB)
DFS Remaining: 11758313472 (10.95 GB)
DFS Used: 266162471 (253.83 MB)
DFS Used%: 2.21%
Replicated Blocks:
	Under replicated blocks: 167
	Blocks with corrupt replicas: 0
	Missing blocks: 82
	Missing blocks (with replication factor 1): 82
	Low redundancy blocks with highest priority to recover: 167
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 172.18.0.4:9866 (project-4-perrito_p4-dn-2.project-4-perrito_p4_default)
Hostname: 352011038fb0
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 266162471 (253.83 MB)
Non DFS Used: 13779799769 (12.83 GB)
DFS Remaining: 11758313472 (10.95 GB)
DFS Used%: 1.03%

In [8]:
import time
time.sleep(30)

In [7]:
import requests
r = requests.get(f"http://boss:9870/webhdfs/v1/single.csv?op=GETFILEBLOCKLOCATIONS")
containerDict = {}
for i in range(len(r.json().get("BlockLocations").get("BlockLocation"))):
    offset = r.json().get("BlockLocations").get("BlockLocation")[i].get("offset")
    status = requests.get(f"http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset={offset}&noredirect=true");
    if (status.status_code == 403):
        id = "lost"
    else:
        id = r.json().get("BlockLocations").get("BlockLocation")[i].get("hosts")[0]
    blocks = r.json().get("BlockLocations").get("BlockLocation")[i].get("length") / 1048576
    count = containerDict.setdefault(id, 0)
    if (count == None):
            count = 0
    containerDict.update({id: (count+blocks)});
containerDict

{'352011038fb0': 85.0, 'lost': 81.83969402313232}

In [8]:
import io
import re

In [11]:
#q10
import requests
count = 0
r = requests.get(f"http://boss:9870/webhdfs/v1/single.csv?op=GETFILEBLOCKLOCATIONS")
for i in range(len(r.json().get("BlockLocations").get("BlockLocation"))):
    offset = r.json().get("BlockLocations").get("BlockLocation")[i].get("offset")
    length = r.json().get("BlockLocations").get("BlockLocation")[i].get("length")
    status = requests.get(f"http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset={offset}&length={length}");
    # print(status)
    if (status.status_code != 403):
        content = io.BytesIO(status.content)
        reader = io.TextIOWrapper(io.BufferedReader(content))
        for i, line in enumerate(reader):
            if re.search(r'Single Family', line):
                count += 1
        # with status.content as f:
        #     reader = io.TextIOWrapper(io.BufferedReader(f))
        #     for i, line in enumerate(reader):
        #         if re.search(r'Single Family', line):
        #             count += 1
count

226958